In [92]:
import csv
import networkx as nx
from networkx.algorithms import community
import copy
from collections import defaultdict
import pickle

# Data Processing

In [93]:
file1_path = "../scraped_data/related_subreddits_clean.tsv"
file2_path = "../scraped_data/related_subreddits_direct_to_file.tsv"

In [94]:
def process_file(file_path):
    """ Gets the subreddit, related subreddit pairs from file"""
    f_content = []
    with open(file_path, encoding ='utf-8') as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')
        for row in reader:
            if len(row) >= 2:
                row[1] = row[1].split(',')[:-1]
                row[1] = [elem.split('/r/')[-1] for elem in row[1]]
            f_content.append(row)
    return f_content

In [95]:
f1_content = process_file(file1_path)
f1_content = f1_content[2:]
f2_content = process_file(file2_path)
all_content = f1_content + f2_content

In [96]:
min_edge_count = 4

all_count = defaultdict(int)
for node in all_content:
    all_count[node[0]] += len(node[1])

all_content_edited = []
for node in all_content:
    if all_count[node[0]] >= min_edge_count:
        all_content_edited.append(node)

print(len(all_content))
print(len(all_content_edited))
all_content = all_content_edited

45404
10008


# Forming Graph From Scraped Data

In [97]:
subr_related_to = defaultdict(set)
for subreddit, related_reddits in all_content:
    subr_related_to[subreddit] = \
    subr_related_to[subreddit].union(set(related_reddits))
print(len(all_content))
print(len(subr_related_to))

edges = []
for subr in subr_related_to:
    for related in subr_related_to[subr]:
        edges.append((subr, related))
print(len(edges))

10008
9979
68675


# Clustering

In [98]:
pickle.dump(edges, open("../cluster_data/ground_truth_edges.pkl", "wb" ))
pickle.dump(list(subr_related_to.keys()), open("../cluster_data/ground_truth_nodes.pkl", "wb" ))

scraped_g = nx.Graph()
scraped_g.add_nodes_from(list(subr_related_to.keys()))
scraped_g.add_edges_from(list(set(edges)))

### Greedy Modularity Clusters

In [99]:
communities_greedy_mod = community.greedy_modularity_communities(scraped_g)
pickle.dump(communities_greedy_mod, open("../cluster_data/greedy_mod_clusters.pkl", "wb" ))
print("Number of communities:", len(communities_greedy_mod))

Number of communities: 211


### Label Propogation Clustering

In [100]:
label_prop_gen = community.label_propagation_communities(scraped_g)
communities_label_prop = [cluster for cluster in label_prop_gen]
pickle.dump(communities_label_prop, open("../cluster_data/label_prop_clusters.pkl", "wb" ))
print("Number of communities:", len(communities_label_prop))

Number of communities: 3027


### Clique Percolation Clustering

In [101]:
def clique_percolation(k):
    communities_clique_gen = community.k_clique_communities(scraped_g, k)
    communities_clique = []
    for cluster in communities_clique_gen:
        communities_clique.append(cluster)
    pickle.dump(communities_clique, \
                open("../cluster_data/clique_k{0}_clusters.pkl".format(k), "wb" ))
    print("k = {0}, number of communities: {1}".format(k, len(communities_clique)))
    return communities_clique

communities_clique_k3 = clique_percolation(3)
communities_clique_k4 = clique_percolation(4)
communities_clique_k5 = clique_percolation(5)

k = 3, number of communities: 1297
k = 4, number of communities: 671
k = 5, number of communities: 110


### Girvan - Newman Clustering

In [ ]:
# # Taking too much time!!
# cluster_generator = community.girvan_newman(scraped_g, most_valuable_edge = None)
# top_level_communities  = next(cluster_generator)
# # next_level_communities = next(communities_generator)
# pickle.dump(top_level_communities, open("../cluster_data/girvan_clusters.pkl", "wb" ))
# print("Number of communities:", len(top_level_communities))

### Fluidity Based Clustering

In [137]:
# Doesn't work because it requires a connected graph
# hyperparameter_k = 3
# communities_fluid = community.asyn_fluidc(scraped_g, k = hyperparameter_k)
# pickle.dump(communities_fluid, open("../cluster_data/fluid_clusters.pkl", "wb" ))
# print("Number of communities:", len(communities_fluid))

In [61]:
def print_results(communities):
    perf = community.quality.performance(scraped_g, communities)
    coverage = community.quality.coverage(scraped_g, communities)
    print("Performance of communities", perf)
    print("Coverage of communities", coverage)
    return None

In [136]:
# While trying ot calculate the performance, jupyter kernel is requiring more
# and more memory, finally the system runs out of memory...
# performance = community.quality.performance(scraped_g, communities_greedy_mod)
coverage = community.quality.coverage(scraped_g, communities_greedy_mod)
modularities = [nx.community.modularity(scraped_g, community) for community in communities_greedy_mod]
print(coverage)
print(sum(modularities) / len(modularities))

0.8988823769933215

In [141]:
coverage = community.quality.coverage(scraped_g, communities_label_prop)
coverage

0.6180591522420608

In [ ]:
# Doesn't work for some reason, says clique percolation clusters
# are not correct partitions
coverage = community.quality.coverage(scraped_g, communities_clique_k5)
coverage